### setup

In [ ]:
import gzip
import json
from typing import List
from collections import Counter, defaultdict
from tqdm import tqdm
import os
import inspect_ai
from openai import OpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from dotenv import load_dotenv
import torch as t
import subprocess
import contextlib
import shutil
import ast
import textwrap
import copy

In [ ]:
from inspect_ai import Task, task
from inspect_ai.dataset import Sample, hf_dataset
from inspect_ai.util import ExecResult, sandbox
from inspect_ai.scorer import CORRECT, INCORRECT, Score, Scorer, Target, accuracy, scorer, stderr
from inspect_ai.solver import TaskState, generate
from inspect_ai.model import get_model
from inspect_ai.log import read_eval_log
import re

In [ ]:
load_dotenv()
login(token = os.environ['HF_TOKEN'])

In [ ]:
IMPORT_HELPER = {
    "python": [
        "import math",
        "import re",
        "import sys",
        "import copy",
        "import datetime",
        "import itertools",
        "import collections",
        "import heapq",
        "import statistics",
        "import functools",
        "import hashlib",
        "import numpy",
        "import numpy as np",
        "import string",
        "from typing import *",
        "from collections import *",
    ],
    "go"    : [
        "math",
        "strings",
        "fmt",
        "strconv",
        "time",
        "bytes",
        "regexp",
        "sort",
        "math/rand",
        "crypto/md5",
    ],
    "cpp"   : [
        "#include<stdlib.h>",
        "#include<algorithm>",
        "#include<math.h>",
        "#include<stdio.h>",
        "#include<vector>",
        "#include<string>",
        "#include<climits>",
        "#include<cstring>",
        "#include<iostream>",
    ],
}

# instruction prepended to code problem
HUMANEVAL_INSTRUCTION = """
Read the following function signature and docstring, and fully implement
the function described. Your response should only contain the code for
this function.\n
"""

LANG_PREFIX = {
    "cpp"          : "// language: C++",
    "java"         : "// language: Java",
    "js"           : "// language: JavaScript",
    "javascript"   : "// language: JavaScript",
    "go"           : "// language: Go",
    "python"       : "# language: Python",
}

In [ ]:
model = get_model(
        'hf/meta-llama/Llama-3.1-8B-Instruct', 
        device = 'auto',
        torch_dtype=t.bfloat16,
)

### Data Processing

In [ ]:
def stream_jsonl_all(filename: str):
    results = []
    fp = gzip.open(open(filename, "rb"), "rt")
    for line in fp:
        if any(not x.isspace() for x in line):
            results.append(json.loads(line))
    fp.close()

    return results

In [ ]:
python_content = stream_jsonl_all('data/python_data.gz')
cpp_content = stream_jsonl_all('data/cpp_data.gz')
go_content = stream_jsonl_all('data/go_data.gz')
java_content = stream_jsonl_all('data/java_data.gz')
js_content = stream_jsonl_all('data/js_data.gz')
content = [python_content, cpp_content, go_content, java_content, js_content]

In [ ]:
generations = stream_jsonl_all('data/python_generations.gz')
generations[0]['generation']

In [ ]:
for lang in content:
    print(lang[0].keys())
    print()

### LLM output => code  
*I need to see LLM output & transform that into code*

In [ ]:
# Installing Java
#     `apt-get update`  
#     `apt-get install -y openjdk-21-jdk`  
# Verify with:  
#     `java -version`  
#     `javac -version` 


In [ ]:
# FIND CODE

def identify_codeblock(completion):
    # remove lang flag
    pattern_1 = re.compile(r"```java\n(.*?)```", re.DOTALL)
    pattern_2 = re.compile(r"```\n(.*?)```", re.DOTALL)
    matches = pattern_1.findall(completion) + pattern_2.findall(completion)
    if len(matches) == 0: raise Exception('error processing model output')

    return matches[0]

def remove_method_header(lines,):
    '''
    deals with three cases
    a) first lines are comments, before method header
    b) first line is method-header
    c) b), but with incorrect indentation? so header had accidentally been removed?
    '''
    removed = False
    og_lines = copy.deepcopy(lines)
    
    while not removed:
        if lines == []:
            return og_lines
        
        line = lines.pop(0)
        if line.lstrip()[:6] == 'public':
            removed = True
        
    return lines

def balance_brackets(lines):
    final_code = '\n'.join(lines)
    difference = final_code.count('{') - final_code.count('}') + 2
    assert difference in [0, 1, 2], 'brackets ain\'t balancing'
    return final_code + ('}' * difference)

def remove_unindented(lines, full_func):
    import_statements = []
    while (not lines[0]) or (lines[0] == lines[0].lstrip()):
        line = lines.pop(0)
        full_func = True
        if 'import' in line: import_statements.append(line)
    
    return lines, import_statements, full_func

def find_code_java(completion: str) -> str:
    code = identify_codeblock(completion)
    lines = code.splitlines()

    lines, import_statements, full_func = remove_unindented(lines, False)

    if full_func or lines[0].lstrip()[:6] == 'public':
        lines = remove_method_header(lines)

    processed_completion = balance_brackets(lines)
    import_statements = '\n'.join(import_statements)
    
    return processed_completion, import_statements

In [ ]:
def get_final_java(imports, state, completion):
    final_code = imports + '\n' + state.metadata['prompt'] + completion + "\n\n" + state.metadata['test'] + "\n"
    return final_code

#### Running Eval

In [ ]:
@scorer(metrics=[accuracy(), stderr()])
def java_scorer() -> Scorer:
    async def score(state: TaskState, target: Target) -> Score:
        task_id = state.sample_id
        idx = int(task_id.split('/')[-1])
        model_completion = state.output.completion
        processed_completion, imports = find_code_java(model_completion)
        final_code = get_final_java(imports, state, processed_completion)

        if 'errormsg' in processed_completion:
            print(f'error in sample: {task_id}')

        tmp_dir = f'/root/srf-project/test_humaneval-x/tmp/test_{idx}/'
        if not os.path.exists(tmp_dir):
            os.makedirs(tmp_dir, exist_ok=True)
        file = os.path.join(tmp_dir, 'Main.java')

        with contextlib.chdir(tmp_dir):
            open(file, 'w').write(final_code)
        
        try:
            compile_proc = subprocess.run(
                ["javac", "Main.java"],
                cwd=tmp_dir,
                capture_output=True,
                text=True,
                timeout=30  
            )
            if compile_proc.returncode != 0:
                print(f"Compilation failed! Task id: {task_id}")
                print("stderr:", compile_proc.stderr)
        except subprocess.TimeoutExpired:
            print("Compilation timed out!")
        except Exception as e:
            print("Compilation error:", e)

        try:
            result = await sandbox().exec(
                cmd=["java", "-cp", tmp_dir, "Main"],
                timeout=30,
            )
        except TimeoutError:
            result = ExecResult(False, 1, "", "Verification timed out.")

        shutil.rmtree(tmp_dir)

        return Score(
            value=CORRECT if result.success else INCORRECT,
            explanation="".join(
                ["The following verification code was executed:\n\n"]
                + [final_code]
                + [f"\nThe submission was incorrect\n\n{result.stderr}"]
                if not result.success
                else [""]
            ),
            metadata={
                'completion': model_completion,
                'processed': processed_completion,
                'final_code': final_code,
                'idx': idx,
                'task_id': task_id,
            },
        )

    return score

In [ ]:
lang = 'java'

def humaneval_record_to_sample(record):
    model_input = HUMANEVAL_INSTRUCTION + LANG_PREFIX[lang] + '\n' + record['prompt'] 
    
    return Sample(
        id=record["task_id"],
        input=model_input,
        target=record["canonical_solution"],
        metadata={
            "prompt": record["prompt"],
            "test": record["test"],
        },
    )

humaneval_dataset = hf_dataset(
    path = 'THUDM/humaneval-x',
    name = lang,
    split = 'test',
    sample_fields = humaneval_record_to_sample,
    trust = True,
)

In [ ]:
samples = 164

@task
def humaneval():
    return Task(
        dataset = humaneval_dataset[:samples],
        solver = generate(),
        scorer = java_scorer(),
        sandbox = 'local',
    )

In [ ]:
epochs = 1
inspect_ai.eval(humaneval(), model = 'openai/gpt-4o-mini', epochs = epochs)

#### Checking outputs

In [ ]:
# 34, 56

idx = 56

log = read_eval_log('/root/srf-project/test_humaneval-x/logs/java_gpt.eval')
data = log.samples[idx].scores['java_scorer'].metadata

In [ ]:
print(data['completion'])

In [ ]:
print(data['final_code'])

In [ ]:
processed = find_code_java(data['completion'])
# state = java_content[2]
# final_code = state['prompt'] + processed + '\n\n' + state['test'] + '\n'

print(processed)

In [ ]:
target_id = 145
idx = 0
model_out = ''

for i, sample in enumerate(log.samples):
    id = int(sample.id.split('/')[-1])
    if id == target_id:
        idx = i
        model_out = sample.messages[-1].content
        print(i)
        print()
        print(model_out)
        break


In [ ]:
print(find_code_new(model_out))